# 量子位相推定

In [28]:
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline

# Qiskitから必要なモジュールをインポート
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.visualization import plot_histogram
from qiskit.circuit import Parameter

## 逆量子フーリエ変換関数を用意

In [29]:
def qft_rotate_single_inv(circuit, i, n):
    if n == 0:
        return circuit
    for qubit in range(0, i):
        circuit.cp(-np.pi/2**(i - qubit), qubit, i)
    circuit.h(i)


def qft_dagger(circuit, n):
    for i in range(math.floor(n/2)):
        circuit.swap(i, n - (i + 1))
    for i in range(n):
        qft_rotate_single_inv(circuit, i , n)

In [30]:
n_encode = 4 # 求めたい固有値の位相角をエンコードする量子ビット数
n_eigstate = 1 # 固有状態の量子ビット数
n = n_encode + n_eigstate
phase = np.random.rand()
theta = Parameter('θ\'')

In [31]:
qc = QuantumCircuit(n, n_encode)
qc.x(n_encode) # 固有状態を用意

for qubit in range(n_encode):
    qc.h(qubit)

repetitions = 1
for count in range(n_encode):
    for i in range(repetitions):
        qc.cp(theta, count, n_encode)
    repetitions *= 2

qc.barrier()
qft_dagger(qc, n_encode)

qc.barrier()
for n in range(n_encode):
    qc.measure(n, n)

In [32]:
# qc.draw(output='mpl) # 量子回路を描画
qc_parametrized = [qc.bind_parameters({theta: phase}) for i in range(2**n_encode - 1)][-1]
backend = Aer.get_backend('qasm_simulator')
shots = 1042
results = execute(qc, backend=backend, shots=shots).result()
answer = results.get_counts()

plot_histogram(answer)

QiskitError: "Mismatch between run_config.parameter_binds and all circuit parameters. Parameter binds: [] Circuit parameters: [ParameterView([Parameter(θ')])]"